# **Object Detection - YOLOR with DeepSORT**

In [ ]:
# GPU 할당 체크
!nvidia-smi

Thu Feb 16 10:39:25 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    22W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## [Step 01] YOLOR 및 DeepSort 설치

In [ ]:
# Clone repository
!git clone https://github.com/hmjeon/YOLOR-Tracking.git

%cd YOLOR-Tracking/yolor_deepsort

# Install
!pip install -qr requirements.txt

%cd ..

Cloning into 'YOLOR-Tracking'...
remote: Enumerating objects: 140, done.
remote: Counting objects: 100% (140/140), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 140 (delta 40), reused 127 (delta 33), pack-reused 0
Receiving objects: 100% (140/140), 1.83 MiB | 12.95 MiB/s, done.
Resolving deltas: 100% (40/40), done.
/content/YOLOR-Tracking/yolor_deepsort
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.8/776.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 102.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 67.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.14.1 requires torch==1.13.1, but you have torch 1.7.0 which is incompatible.
torchaudio 0.13.1+cu116 requires torch==1.13.1, but you have torch 1.7.0 wh

## [Step 02] 데이터세트 다운로드

In [ ]:
!wget -O dataset.zip https://www.dropbox.com/s/0v2ry2jbqaes64o/dataset.zip?dl=1
!unzip dataset.zip -d dataset

--2023-02-16 10:45:14--  https://www.dropbox.com/s/0v2ry2jbqaes64o/dataset.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/0v2ry2jbqaes64o/dataset.zip [following]
--2023-02-16 10:45:14--  https://www.dropbox.com/s/dl/0v2ry2jbqaes64o/dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc485d619625abfbf827721785c5.dl.dropboxusercontent.com/cd/0/get/B2noP4lldu6E45wHsR0FxwpOdDqeOLKfIoV95M_7kbRnlzvKZ4Or0bwerl1gy0YabVWecIDwRBVXWbB-p_fjrELY4anzxqOAsHM-mprap9GtryWAa1cjZdlhNuYJIDKsb7MqDsIjqbSwxbfqU-BiMESdiwolPQzhntgS3yOTotBFag/file?dl=1# [following]
--2023-02-16 10:45:14--  https://uc485d619625abfbf827721785c5.dl.dropboxusercontent.com/cd/0/get/B2noP4lldu6E45wHsR0FxwpOdDqeOLKfIoV95M_7kbRnlzvKZ4Or0bwerl1gy0YabVWecIDwRBVXWbB-p

## [Step 03] Weight 및 Checkpoint 다운로드

In [ ]:
# YOLOR-P6, W6, E6, D6
!wget -O yolor_p6.pt https://www.dropbox.com/s/ldyk3dzuuu3y9xx/yolor_p6.pt?dl=1
#!wget -O yolor_w6.pt https://www.dropbox.com/s/cqrxp1tlfk7sb51/yolor_w6.pt?dl=1
#!wget -O yolor_d6.pt https://www.dropbox.com/s/znl5pfwcyr8ylcb/yolor-d6.pt?dl=1
#!wget -O yolor_e6.py https://www.dropbox.com/s/ql8e89jwi6wewbc/yolor-e6.pt?dl=1

# Download checkpoint for DeepSORT
!wget -O ckpt.t7 https://www.dropbox.com/s/3molk6h70dsiiap/ckpt.t7?dl=1
!mv ckpt.t7 yolor_deepsort/deep_sort_pytorch/deep_sort/deep/checkpoint

--2023-02-16 10:43:55--  https://www.dropbox.com/s/ldyk3dzuuu3y9xx/yolor_p6.pt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/ldyk3dzuuu3y9xx/yolor_p6.pt [following]
--2023-02-16 10:43:55--  https://www.dropbox.com/s/dl/ldyk3dzuuu3y9xx/yolor_p6.pt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc3c897facac709495c1f0a5f00c.dl.dropboxusercontent.com/cd/0/get/B2n0RTEp82tfJRjzzSoPVdTlK2p9wJFJqq5PkuaoJyJHpOVqcX9bo4t8-XXS3TSNV4ikNxicVY17Wb_CedF96rHav-JPSHCG6RajruOa4H1ccJQKJML2fTzi0jD23fpCC8IrIgmAZJybEYt6kt44SIOOXhqMqjSQj756fIgYR9_pgg/file?dl=1# [following]
--2023-02-16 10:43:56--  https://uc3c897facac709495c1f0a5f00c.dl.dropboxusercontent.com/cd/0/get/B2n0RTEp82tfJRjzzSoPVdTlK2p9wJFJqq5PkuaoJyJHpOVqcX9bo4t8-XXS3TSNV4ikNxicVY17Wb_Ce

## [Step 04] YOLOR DeepSORT 추론

In [ ]:
in_path = "dataset/videos/"
out_path = "dataset/videos_out/"

!python yolor_deepsort/detect_track_trails.py \
                --source $in_path \
                --output $out_path \
                --cfg yolor_deepsort/cfg/yolor_p6.cfg \
                --weights yolor_p6.pt \
                --names yolor_deepsort/data/coco.names \
                --data yolor_deepsort/deep_sort_pytorch/configs/deep_sort.yaml \
                --conf 0.25 \
                --img-size 1280 \
                --device 0 # --view-img

Namespace(agnostic_nms=False, augment=False, cfg='yolor_deepsort/cfg/yolor_p6.cfg', classes=None, conf_thres=0.25, data='yolor_deepsort/deep_sort_pytorch/configs/deep_sort.yaml', device='0', img_size=1280, iou_thres=0.5, names='yolor_deepsort/data/coco.names', output='dataset/videos_out/', save_txt=False, source='dataset/videos/', trailslen=64, update=False, view_img=False, weights=['yolor_p6.pt'])
video 1/3 (1/249) /content/YOLOR-Tracking/dataset/videos/test_01.mp4: 768x1280 1 persons, 6 cars, 1 trucks, Done. (0.052s)
video 1/3 (2/249) /content/YOLOR-Tracking/dataset/videos/test_01.mp4: 768x1280 1 persons, 6 cars, 2 trucks, Done. (0.051s)
video 1/3 (3/249) /content/YOLOR-Tracking/dataset/videos/test_01.mp4: 768x1280 1 persons, 7 cars, 3 trucks, Done. (0.036s)
video 1/3 (4/249) /content/YOLOR-Tracking/dataset/videos/test_01.mp4: 768x1280 1 persons, 8 cars, 2 trucks, Done. (0.036s)
video 1/3 (5/249) /content/YOLOR-Tracking/dataset/videos/test_01.mp4: 768x1280 1 persons, 10 cars, 3 truck

## [Step 05] YOLOR DeepSORT 추론 (Custom 데이터셋)

In [ ]:
in_path = "dataset/eggs/videos/"
out_path = "dataset/eggs/videos_out/"

!python yolor_deepsort/detect_track_trails.py \
                --source $in_path \
                --output $out_path \
                --cfg ../YOLOR/yolor_dice.cfg \
                --weights ../YOLOR/yolor_dice.pt \
                --names ../YOLOR/data_dice.names \
                --data yolor_deepsort/deep_sort_pytorch/configs/deep_sort.yaml \
                --conf 0.4 \
                --img-size 1280 \
                --device 0 # --view-img